In [1]:
from gensim.models.keyedvectors import KeyedVectors

# load word vectors
word_vectors = KeyedVectors.load_word2vec_format(
    'word2vec')

# create list of projection data
# tuple (label, vector)
labels = word_vectors.index_to_key
vectors = [word_vectors[label] for label in labels]

projection_data = [(label, vector) for label, vector in zip(labels, vectors)]

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorboard.plugins import projector

def create_projection(projection_data, path='./tensorboard/'):
    meta_file = 'metadata.tsv'
    samples = len(projection_data)
    vector_dim = len(projection_data[0][1])
    projection_matrix = np.zeros((samples, vector_dim))

    # write meta file with labels, create projection_matrix
    with open(os.path.join(path, meta_file), 'w') as f:
        for i, row in enumerate(projection_data):
            label, vector = row[0], row[1]
            projection_matrix[i] = vector
            f.write(f"{label}\n")

    weights = tf.Variable(
        projection_matrix, trainable=False, name='word_embeddings'
    )

    checkpoint = tf.train.Checkpoint(embedding=weights)
    checkpoint.save(os.path.join(path, "embedding.ckpt"))
    writer = tf.summary.create_file_writer(path)
    
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = meta_file
    projector.visualize_embeddings(path, config)

create_projection(projection_data)

2023-07-25 12:14:17.388721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-25 12:14:17.388783: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-25 12:14:43.951452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-25 12:14:43.951519: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-25 12:14:43.951568: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (august-OptiPlex-3040): /proc/driver/nvidia/version does not exist
2023-07-25 12:14:43.961526: I tensorflow/core/platform/c

In [4]:
%load_ext tensorboard
%tensorboard --logdir="./tensorboard/"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4701), started 0:00:26 ago. (Use '!kill 4701' to kill it.)